In [20]:
import pandas as pd
import numpy as np
from datetime import datetime

excel_obj = pd.ExcelFile('mock_isp_data.xlsx')
sheets_name = excel_obj.sheet_names

df = excel_obj.parse('Main_Data')

df['MSISDN'] = df['MSISDN'].apply(str)

date_cols = [
    'CONTRACT_START_DT',
    'CONTRACT_END_DT'
]

df[date_cols] = df[date_cols].apply(pd.to_datetime)

todays = pd.Timestamp(datetime.now().date())

df['REMAIN_DAYS'] = (df['CONTRACT_END_DT'] - todays).dt.days

df['REMAIN_VALUE'] = df['REMAIN_DAYS'] * (df['RC_RATE'] / 30)

df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'REMAIN_VALUE']]

,CUST_FULL_NAME,REMAIN_DAYS,REMAIN_VALUE
0,นายหมดอายุ แล้วครับ,-49,-978.366667
1,บริษัท กำลังจะหมด (15วัน),9,269.700000
2,ร้านค้า ยังเหลือเยอะ,316,12640.000000
3,นายพอดีเป๊ะ (วันนี้หมด),-6,-89.800000
4,บจก. เพิ่งเริ่มสัญญา,681,6787.300000


In [21]:
conditions = [
    (df['REMAIN_DAYS'] < 0),
    (df['REMAIN_DAYS'] >= 0) & (df['REMAIN_DAYS'] <= 30),
    (df['REMAIN_DAYS'] > 30)
]

choices = ['EXPIRED', 'WARNING', 'HEALTHY']

df['CONTRACT_STATUS'] = np.select(conditions, choices, default='UNKNOWN')

display(df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'CONTRACT_STATUS']])

,CUST_FULL_NAME,REMAIN_DAYS,CONTRACT_STATUS
0,นายหมดอายุ แล้วครับ,-49,EXPIRED
1,บริษัท กำลังจะหมด (15วัน),9,WARNING
2,ร้านค้า ยังเหลือเยอะ,316,HEALTHY
3,นายพอดีเป๊ะ (วันนี้หมด),-6,EXPIRED
4,บจก. เพิ่งเริ่มสัญญา,681,HEALTHY


In [22]:
status_summary = df.groupby('CONTRACT_STATUS').size()

print('สรุปจำนวนลูกค้าตามสถานะ: ')
display(status_summary)

status_df = status_summary.reset_index(name='CUSTOMER_COUNT')
display(status_df)

สรุปจำนวนลูกค้าตามสถานะ: 


CONTRACT_STATUS
EXPIRED    2
HEALTHY    2
WARNING    1
dtype: int64

,CONTRACT_STATUS,CUSTOMER_COUNT
0,EXPIRED,2
1,HEALTHY,2
2,WARNING,1


In [23]:
report = df.groupby('CONTRACT_STATUS')['RC_RATE'].agg(['count', 'sum', 'mean'])

report = report.rename(columns={
    'count': 'จำนวนลูกค้า (ราย)',
    'sum': 'ยอดเงินรวม (บาท)',
    'mean': 'เฉลี่ยตัวหัว (บาท)'
})

styled_report = report.style.format({
    'ยอดเงินรวม (บาท)': '{:,.2f}',
    'เฉลี่ยตัวหัว (บาท)': '{:,.2f}'
})

display(styled_report)

,จำนวนลูกค้า (ราย),ยอดเงินรวม (บาท),เฉลี่ยตัวหัว (บาท)
CONTRACT_STATUS,,,
EXPIRED,2,"1,048.00",524.00
HEALTHY,2,"1,499.00",749.50
WARNING,1,899.00,899.00


In [24]:
report.to_excel('isp_summary_report.xlsx')

df.to_excel('isp_processed_data.xlsx')

print('บันทึกไฟล์เรียบร้อย')

บันทึกไฟล์เรียบร้อย
